# Deep Universal Regular Conditional Expectations:

---
This implements the universal deep neural model of $\mathcal{NN}_{1_{\mathbb{R}^n},\mathcal{D}}^{\sigma:\star}$ [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

## What does this code do?
1. Learn Heteroskedastic Non-Linear Regression Problem
     - $Y\sim f_{\text{unkown}}(x) + \epsilon$ where $f$ is an known function and $\epsilon\sim Laplace(0,\|x\|)$
2. Learn Random Bayesian Network's Law:
    - $Y = W_J Y^{J-1}, \qquad Y^{j}\triangleq \sigma\bullet A^{j}Y^{j-1} + b^{j}, \qquad Y^0\triangleq x$

3. In the above example if $A_j = M_j\odot \tilde{A_j}$ where $\tilde{A}_j$ is a deterministic matrix and $M_j$ is a "mask", that is, a random matrix with binary entries and $\odot$ is the Hadamard product then we recover the dropout framework.
4. Learn the probability distribution that the unique strong solution to the rough SDE with uniformly Lipschitz drivers driven by a factional Brownian motion with Hurst exponent $H \in [\frac1{2},1)$:
$$
X_t^x = x + \int_0^t \alpha(s,X_s^x)ds + \int_0^t \beta(s,X_s^x)dB_s^H
$$
belongs, at time $t=1$, to a ball about the initial point $x$ of random radius given by an independant exponential random-variable with shape parameter $\lambda=2$
5. Train a DNN to predict the returns of bitcoin with GD.  Since this has random initialization then each prediction of a given $x$ is stochastic...We learn the distribution of this conditional RV (conditioned on x in the input space).
$$
Y_x \triangleq \hat{f}_{\theta_{T}}(x), \qquad \theta_{(t+1)}\triangleq \theta_{(t)} + \lambda \sum_{x \in \mathbb{X}} \nabla_{\theta}\|\hat{f}_{\theta_t}(x) - f(x)\|, \qquad \theta_0 \sim N_d(0,1);
$$
$T\in \mathbb{N}$ is a fixed number of "SGD" iterations (typically identified by cross-validation on a single SGD trajectory for a single initialization) and where $\theta \in \mathbb{R}^{(d_{J}+1)+\sum_{j=0}^{J-1} (d_{j+1}d_j + 1)}$ and $d_j$ is the dimension of the "bias" vector $b_j$ defining each layer of the DNN with layer dimensions:
$$
\hat{f}_{\theta}(x)\triangleq A^{(J)}x^{(J)} + b^{(J)},\qquad x^{(j+1)}\triangleq \sigma\bullet A^{j}x^{(j)} + b^{j},\qquad x^{(0)}\triangleq x
.
$$

6. Extreme Learning Machines: 
    Just like the Bayesian network but then last layer is trained on the training set using KRidge!

#### Mode:
Software/Hardware Testing or Real-Deal?

In [1]:
trial_run = True

### Simulation Method:

In [2]:
# Random DNN
# f_unknown_mode = "Heteroskedastic_NonLinear_Regression"

# Random DNN internal noise
## Real-world data version
f_unknown_mode = "Extreme_Learning_Machine"
### General Parameters
# activation_function == 'thresholding'
activation_function = 'sigmoid'
### Dataset Option 1
dataset_option = 'SnP'
### Dataset Option 2
# dataset_option = 'crypto'
Depth_Bayesian_DNN = 1
N_Random_Features = 10**3
## Simulated Data version
# f_unknown_mode = "DNN_with_Random_Weights"
width = 10**3

# Random Dropout applied to trained DNN
# f_unknown_mode = "DNN_with_Bayesian_Dropout"
Dropout_rate = 0.75

# GD with Randomized Input
# f_unknown_mode = "GD_with_randomized_input"
# GD_epochs = 50

# SDE with fractional Driver
f_unknown_mode = "Rough_SDE"
N_Euler_Steps = 50
Hurst_Exponent = 0.51

f_unknown_mode = "Rough_SDE_Vanilla"
## Define Process' dynamics in (2) cell(s) below.

---
#### More Meta-Parameters for "Vanilla" fractional SDE

In [3]:
## Monte-Carlo
N_Euler_Maruyama_Steps = 50
N_Monte_Carlo_Samples = 10**2
N_Monte_Carlo_Samples_Test = 10**1 # How many MC-samples to draw from test-set?

# End times for Time-Grid
T_end = 1
T_end_test = 1.1


## Grid
N_Grid_Finess = 5
Max_Grid = 0.5
x_0 = 1

# Number of Centers (\hat{\mu}_s)
N_Quantizers_to_parameterize = 10
Hurst_Exponent = 0.5
uniform_noise_level = 0

# Hyper-parameters of Cover
delta = 0.1
N_measures_per_center = 100

#### Vanilla Drift

In [4]:
def alpha(t,x):
    return .1*(.1-.5*(.01**2))*t + np.cos(x)

#### Vanilla Vol

In [5]:
def beta(t,x):
    return 0.01+t*np.cos(x)

---

## Problem Dimension

In [6]:
problem_dim = 2
if f_unknown_mode != 'Extreme_Learning_Machine':
    width = int(2*(problem_dim+1))

#### Vanilla fractional SDE:
If f_unknown_mode == "Rough_SDE_Vanilla" is selected, then we can specify the process's dynamics.  

In [7]:
# Depricated
# #--------------------------#
# # Define Process' Dynamics #
# #--------------------------#
# drift_constant = 0.1
# volatility_constant = 0.01

# # Define DNN Applier
# def f_unknown_drift_vanilla(x):
# #     x_internal = drift_constant*np.ones(problem_dim)
#     x_internal = drift_constant*np.sin(x)
#     return x_internal
# def f_unknown_vol_vanilla(x):
# #     x_internal = volatility_constant*np.diag(np.ones(problem_dim))
#     x_internal = volatility_constant*np.diag(np.cos(x))
#     return x_internal

## Note: *Why the procedure is so computationally efficient*?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

#### Grid Hyperparameter(s)
- Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.
- Number of Training Points to Generate

In [8]:
train_test_ratio = .1
N_train_size = 100

Monte-Carlo Paramters

In [9]:
## Monte-Carlo
N_Monte_Carlo_Samples = 10**3

Initial radis of $\delta$-bounded random partition of $\mathcal{X}$!

In [10]:
# Hyper-parameters of Cover
delta = 0.1
Proportion_per_cluster = .1

## Dependencies and Auxiliary Script(s)

In [11]:
# %run Loader.ipynb
exec(open('Loader.py').read())
# Load Packages/Modules
exec(open('Init_Dump.py').read())
import time as time #<- Note sure why...but its always seems to need 'its own special loading...'

Using TensorFlow backend.


Deep Feature Builder - Ready
Deep Classifier - Ready
Deep Feature Builder - Ready


# Simulate or Parse Data

In [12]:
#-------------------------------------------------------#
print("Generating/Prasing Data and Training MC-Oracle")
#-------------------------------------------------------#
if (f_unknown_mode != 'Rough_SDE') and (f_unknown_mode != 'Rough_SDE_Vanilla'):
    # %run Data_Simulator_and_Parser.ipynb
    exec(open('Data_Simulator_and_Parser.py').read())
else:
    # Renaming Some internal Parameter(s)
    groud_truth = "rSDE"
    test_size_ratio = train_test_ratio
    Ratio_fBM_to_typical_vol = 1
    output_dim = problem_dim
    T_begin = 0
    T_end = 1
    # Run da code
    %run Fractional_SDE/fractional_SDE_Simulator.ipynb
    %run Fractional_SDE/Data_Simulator_and_Parser___fractional_SDE.ipynb
#     exec(open('Fractional_SDE/fractional_SDE_Simulator.py').read())
#     exec(open('Fractional_SDE/Data_Simulator_and_Parser___fractional_SDE.py').read())

# Verbosity is nice
print("Generated Data:")
print("Number of Training Datums:"+str(X_train.shape[0]))
print("Number of Testing Datums:"+str(X_test.shape[0]))

  0%|          | 0/10 [00:00<?, ?it/s]

Generating/Prasing Data and Training MC-Oracle
Building Training + Testing Set - rough-SDE Ground-Truth



100%|██████████| 1/1 [00:10<00:00, 10.32s/it]

Generated Data:
Number of Training Datums:50
Number of Testing Datums:50


### Rescale

In [13]:
# Rescale
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Run: Main (DNM) and Oracle Models

In [14]:
print("------------------------------")
print("Running script for main model!")
print("------------------------------")
%run Universal_Measure_Valued_Networks_Backend.ipynb
# exec(open('Universal_Measure_Valued_Networks_Backend.py').read())

print("------------------------------------")
print("Done: Running script for main model!")
print("------------------------------------")

100%|██████████| 5/5 [00:00<00:00, 13609.03it/s]

------------------------------
Running script for main model!
------------------------------
Deep Feature Builder - Ready
Deep Classifier - Ready
Training Classifer Portion of Type-A Model
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.5s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.5s finished


Epoch 1/2
4/4 [==============================] - 0s 1ms/step - loss: 1.6083 - accuracy: 0.2200
Epoch 2/2
4/4 [==============================] - 0s 905us/step
Training Classifer Portion of Type Model: Done!
------------------------------------
Done: Running script for main model!
------------------------------------


### Evaluate Main and Oracle Model's Performance

In [15]:
%run Universal_Measure_Valued_Networks_Backend_EVALUATOR.ipynb
# exec(open(Universal_Measure_Valued_Networks_Backend_EVALUATOR.py).read())

  0%|          | 0/50 [00:00<?, ?it/s]

#--------------------#
 Get Training Error(s)
#--------------------#


  8%|▊         | 4/50 [00:00<00:01, 37.49it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#----------------#
 Get Test Error(s)
#----------------#


100%|██████████| 50/50 [00:00<00:00, 52.52it/s]


#------------------------#
 Get Testing Error(s): END
#------------------------#
                                     DNM  MC-Oracle
W1-95L                          7.418265   0.000000
W1                              8.417237   0.000000
W1-95R                          9.337770   0.000000
M-95L                           0.283303   0.000000
M                               0.343522   0.000000
M-95R                           0.400752   0.000000
N_Par                          23.000000   0.000000
Train_Time                      7.625761   0.944898
Test_Time/MC-Oracle_Test_Time   0.055351   1.000000


## 1) *Pointmass Benchmark(s)*
These benchmarks consist of subsets of $C(\mathbb{R}^d,\mathbb{R})$ which we lift to models in $C(\mathbb{R}^d,\cap_{1\leq q<\infty}\mathscr{P}_{q}(\mathbb{R}))$ via:
$$
\mathbb{R}^d \ni x \to f(x) \to \delta_{f(x)}\in \cap_{1\leq q<\infty}\mathcal{P}_{q}(\mathbb{R}).
$$

In [17]:
# # Transport-problem initializations #
# #-----------------------------------#
# if output_dim != 1:
#     ## Multi-dimensional
#     # Externally Update Empirical Weights for multi-dimensional case
#     empirical_weights = np.full((N_Monte_Carlo_Samples,),1/N_Monte_Carlo_Samples)
#     # Also Initialize
#     Sinkhorn_regularization = 0.1
# else:
#     ## Single-Dimensional
#     # Initialize Empirical Weights
#     empirical_weights = (np.ones(N_Monte_Carlo_Samples)/N_Monte_Carlo_Samples).reshape(-1,)

# #-------------------------#
# # Define Transport Solver #
# #-------------------------#
# def transport_dist(x_source,w_source,x_sink,w_sink,output_dim,OT_method="Sliced"):
#     # Decide which problem to solve (1D or multi-D)?
#     if output_dim == 1:
#         OT_out = ot.emd2_1d(x_source,
#                             x_sink,
#                             w_source,
#                             w_sink)
#     else:
#         # COERCSION
#         ## Update Source Distribution
#         x_source = points_of_mass.reshape(-1,output_dim)
#         ## Update Sink Distribution
#         x_sink = np.array(Y_train[i,]).reshape(-1,output_dim)
        
#         if OT_method == "Sinkhorn":
#             OT_out = ot.bregman.empirical_sinkhorn2(X_s = x_source, 
#                                                     X_t = x_sink,
#                                                     a = w_source, 
#                                                     b = w_sink, 
#                                                     reg=0.01, 
#                                                     verbose=False,
#                                                     method = "sinkhorn_stabilized")
#             # COERSION
#             OT_out = float(OT_out[0])
#         else:
#             OT_out = ot.sliced.sliced_wasserstein_distance(X_s = x_source, 
#                                                     X_t = x_sink,
#                                                     a = w_source, 
#                                                     b = w_sink, 
#                                                     seed = 2020)
#             # COERSION
#             OT_out = float(OT_out)
#     # Return (regularized?) Transport Distance
#     return OT_out

In [18]:
exec(open('CV_Grid.py').read())
# Notebook Mode:
# %run Evaluation.ipynb
# %run Benchmarks_Model_Builder_Pointmass_Based.ipynb
# Terminal Mode (Default):
exec(open('Evaluation.py').read())
exec(open('Benchmarks_Model_Builder_Pointmass_Based.py').read())

  0%|          | 0/2 [00:00<?, ?it/s]

Deep Feature Builder - Ready
--------------
Training: ENET
--------------


 20%|██        | 10/50 [00:00<00:00, 97.53it/s]

---------------------
Training: ENET - Done
---------------------
#------------#
 Get Error(s) 
#------------#


 22%|██▏       | 11/50 [00:00<00:00, 99.83it/s]

...

#------------#
 Get Error(s) 
#------------#


 20%|██        | 10/50 [00:00<00:00, 95.20it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#


100%|██████████| 50/50 [00:00<00:00, 94.37it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame
                                     DNM  MC-Oracle         ENET       KRidge  \
W1-95L                          7.614287   0.000000            -            -   
W1                              8.417237   0.000000            -            -   
W1-95R                          9.337770   0.000000            -            -   
M-95L                           0.416545   0.000000    0.0920916  8.41396e+42   
M                               0.468905   0.000000     0.121827  8.41396e+42   
M-95R                           0.531640   0.000000     0.148096  8.41396e+42   
N_Par                          23.000000   0.000000          106            0   
Train_Time                      7.625761   0.944898  1.62084e+09       1.6049   
Test_Time/MC-Oracle_Test_Time   0.055351   1.000000   0.00012336  0.000450491   

                                      GBRF          DNN  
W1-95L                                   -           

# Summary of Point-Mass Regression Models

#### Training Model Facts

In [19]:
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

                                     DNM  MC-Oracle         ENET       KRidge  \
W1-95L                          7.614287   0.000000            -            -   
W1                              8.417237   0.000000            -            -   
W1-95R                          9.337770   0.000000            -            -   
M-95L                           0.416545   0.000000    0.0920916  8.41396e+42   
M                               0.468905   0.000000     0.121827  8.41396e+42   
M-95R                           0.531640   0.000000     0.148096  8.41396e+42   
N_Par                          23.000000   0.000000          106            0   
Train_Time                      7.625761   0.944898  1.62084e+09       1.6049   
Test_Time/MC-Oracle_Test_Time   0.055351   1.000000   0.00012336  0.000450491   

                                      GBRF          DNN  
W1-95L                                   -            -  
W1                                       -            -  
W1-95R         

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN
W1-95L,7.614287,0.000000,-,-,-,-
W1,8.417237,0.000000,-,-,-,-
W1-95R,9.337770,0.000000,-,-,-,-
M-95L,0.416545,0.000000,0.0920916,8.41396e+42,0.344304,6.48093e+11
M,0.468905,0.000000,0.121827,8.41396e+42,0.42216,6.48093e+11
M-95R,0.531640,0.000000,0.148096,8.41396e+42,0.473096,6.48093e+11
N_Par,23.000000,0.000000,106,0,924,14
Train_Time,7.625761,0.944898,1.62084e+09,1.6049,1.16024,2.93983
Test_Time/MC-Oracle_Test_Time,0.055351,1.000000,0.00012336,0.000450491,0.000324029,0.052523


#### Testing Model Facts

In [20]:
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

                                     DNM  MC-Oracle          ENET  \
W1-95L                          7.418265   0.000000  9.470811e+00   
W1                              8.417237   0.000000  1.130337e+01   
W1-95R                          9.337770   0.000000  1.344438e+01   
M-95L                           0.283303   0.000000  9.209162e-02   
M                               0.343522   0.000000  1.218272e-01   
M-95R                           0.400752   0.000000  1.480957e-01   
N_Par                          23.000000   0.000000  1.060000e+02   
Train_Time                      7.625761   0.944898  1.620839e+09   
Test_Time/MC-Oracle_Test_Time   0.055351   1.000000  1.233601e-04   

                                     KRidge        GBRF           DNN  
W1-95L                         4.920840e+42    8.961968  3.274312e+11  
W1                             4.975703e+42   11.311635  3.311580e+11  
W1-95R                         5.056688e+42   14.036639  3.354489e+11  
M-95L                

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN
W1-95L,7.418265,0.000000,9.470811e+00,4.920840e+42,8.961968,3.274312e+11
W1,8.417237,0.000000,1.130337e+01,4.975703e+42,11.311635,3.311580e+11
W1-95R,9.337770,0.000000,1.344438e+01,5.056688e+42,14.036639,3.354489e+11
M-95L,0.283303,0.000000,9.209162e-02,8.413963e+42,0.259302,6.480934e+11
M,0.343522,0.000000,1.218272e-01,8.413963e+42,0.306053,6.480934e+11
M-95R,0.400752,0.000000,1.480957e-01,8.413963e+42,0.350270,6.480934e+11
N_Par,23.000000,0.000000,1.060000e+02,0.000000e+00,924.000000,1.400000e+01
Train_Time,7.625761,0.944898,1.620839e+09,1.604898e+00,1.160245,2.939826e+00
Test_Time/MC-Oracle_Test_Time,0.055351,1.000000,1.233601e-04,4.504912e-04,0.000324,5.252303e-02


## 2) *Gaussian Benchmarks*

- Bencharm 1: [Gaussian Process Regressor](https://scikit-learn.org/stable/modules/gaussian_process.html)
- Benchmark 2: Deep Gaussian Networks:
These models train models which assume Gaussianity.  We may view these as models in $\mathcal{P}_2(\mathbb{R})$ via:
$$
\mathbb{R}^d \ni x \to (\hat{\mu}(x),\hat{\Sigma}(x)\hat{\Sigma}^{\top})\triangleq f(x) \in \mathbb{R}\times [0,\infty) \to 
(2\pi)^{-\frac{d}{2}}\det(\hat{\Sigma}(x))^{-\frac{1}{2}} \, e^{ -\frac{1}{2}(\cdot - \hat{\mu}(x))^{{{\!\mathsf{T}}}} \hat{\Sigma}(x)^{-1}(\cdot - \hat{\mu}(x)) } \mu \in \mathcal{G}_d\subset \mathcal{P}_2(\mathbb{R});
$$
where $\mathcal{G}_1$ is the set of Gaussian measures on $\mathbb{R}$ equipped with the relative Wasserstein-1 topology.

Examples of this type of architecture are especially prevalent in uncertainty quantification; see ([Deep Ensembles](https://arxiv.org/abs/1612.01474)] or [NOMU: Neural Optimization-based Model Uncertainty](https://arxiv.org/abs/2102.13640).  Moreover, their universality in $C(\mathbb{R}^d,\mathcal{G}_2)$ is known, and has been shown in [Corollary 4.7](https://arxiv.org/abs/2101.05390).

In [ ]:
# %run Benchmarks_Model_Builder_Mean_Var.ipynb
exec(open('Benchmarks_Model_Builder_Mean_Var.py').read())

In [ ]:
print("Prediction Quality (Updated): Test")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

In [ ]:
print("Prediction Quality (Updated): Train")
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

# 3) The natural Universal Benchmark: [Bishop's Mixture Density Network](https://publications.aston.ac.uk/id/eprint/373/1/NCRG_94_004.pdf)

This implementation is as follows:
- For every $x$ in the trainingdata-set we fit a GMM $\hat{\nu}_x$, using the [Expectation-Maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm), with the same number of centers as the deep neural model in $\mathcal{NN}_{1_{\mathbb{R}^d},\mathcal{D}}^{\sigma:\star}$ which we are evaluating.  
- A Mixture density network is then trained to predict the infered parameters; given any $x \in \mathbb{R}^d$.

In [ ]:
if output_dim == 1:
    # %run Mixture_Density_Network.ipynb
    exec(open('Mixture_Density_Network.py').read())

## Get Final Outputs
Now we piece together all the numerical experiments and report a nice summary.

---
# Final Results
---

## Prasing Quality Metric Results

#### Finalizing Saving
**Note:** *We do it in two steps since the grid sometimes does not want to write nicely...*

In [ ]:
## Write Performance Metrics
### Incase caption breaks
Summary_pred_Qual_models.to_latex((results_tables_path+"/Final_Results/"+"Performance_metrics_Problem_Type_"+str(f_unknown_mode)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS.tex"),
                                 float_format="{:0.3g}".format)
text_file = open((results_tables_path+"/Final_Results/"+"ZZZ_CAPTION_Performance_metrics_Problem_Type_"+str(f_unknown_mode)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS___CAPTION.tex"), "w")
text_file.write("Quality Metrics; d:"+str(problem_dim)+", D:"+str(output_dim)+", Depth:"+str(Depth_Bayesian_DNN)+", Width:"+str(width)+", Dropout rate:"+str(Dropout_rate)+".")
text_file.close()


### Incase caption does not break
Summary_pred_Qual_models.to_latex((results_tables_path+"/Final_Results/"+"Performance_metrics_Problem_Type_"+str(f_unknown_mode)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS.tex"),
                                 caption=("Quality Metrics; d:"+str(problem_dim)+", D:"+str(output_dim)+", Depth:"+str(Depth_Bayesian_DNN)+", Width:"+str(width)+", Dropout rate:"+str(Dropout_rate)+"."),
                                 float_format="{:0.3g}".format)

# For Terminal Runner(s):

In [ ]:
# For Terminal Running
print("===================")
print("Predictive Quality:")
print("===================")
print(Summary_pred_Qual_models)
print("===================")
print(" ")
print(" ")
print(" ")
print("Kernel_Used_in_GPR: "+str(GPR_trash.kernel))
print("🙃🙃 Have a wonderful day! 🙃🙃")
Summary_pred_Qual_models

---
# Fin
---

---